In [ ]:
%%writefile content.json

{"intents": [
        {"tag": "greeting",
         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up"],
         "responses": ["Hello!", "Good to see you again!", "Hi there, how can I help you?"],
         "context_set": ""
        },
        {"tag": "goodbye",
         "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day", "bye", "Good Night"],
         "responses": ["Sad to see you go", "Talk to you later", "Goodbye!", "Bye", "Goodbye"],
         "context_set": ""
        },
        {"tag": "age",
         "patterns": ["how old", "how old is bot", "what is your age", "how old are you", "age?"],
         "responses": ["I am 2 days old!", "2 days young!"],
         "context_set": ""
        },
        {"tag": "brac1",
         "patterns": ["What is bracu", "BRACU is what", "Where is BRACU", "Do you know bracu", "BRAC University", "BRACU"],
         "responses": ["BRAC University (also known as BracU) is a private research university located at 66 Mohakhali, Dhaka, Bangladesh. It was established in 2001 as a branch of Sir Fazle Hasan Abed's BRAC under the Private University Act 1992. [Source: Wikipedia]"],
         "context_set": ""
        },
        {"tag": "brac2",
         "patterns": ["What is brac", "BRAC is what", "Where is BRAC", "Do you know brac", "BRAC"],
         "responses": ["BRAC is an international development organisation based in Bangladesh. In order to receive foreign donations, BRAC was subsequently registered under the NGO Affairs Bureau of the Government of Bangladesh. [Source: Wikipedia]"],
         "context_set": ""
        },
        {"tag": "brac3",
         "patterns": ["What is full form brac", "BRAC Full Form"],
         "responses": ["Originally founded in 1972 as the Bangladesh Rehabilitation Assistance Committee and later known as the Bangladesh Rural Advancement Committee", "Bangladesh Rehabilitation Assistance Committee"],
         "context_set": ""
        },
        {"tag": "tarc1",
         "patterns": ["What is full form of TARC", "TARC Full Form", "tarc is what", "Where is tarc", "Do you know tarc", "TARC"],
         "responses": ["TARC stands for Training And Resource Centre. This is where BRAC University sends students to their campus in Savar for a real life experience"],
         "context_set": "" 
        },
        {"tag": "name",
         "patterns": ["what is your name", "what should I call you", "whats your name?"],
         "responses": ["You can call me bro, BRACU BRO", "I'm BRACU BRO!"],
         "context_set": ""
        },
        {"tag": "Scholar1",
         "patterns": ["How many types of scholarships are provided by Brac","Where can I get information about scholarship", "Scholarship"],
         "responses": ["Click this link https://www.bracu.ac.bd/admissions/scholarships-financial-aid","https://www.bracu.ac.bd/admissions/scholarships-financial-aid","https://www.bracu.ac.bd/admissions/scholarships-financial-aid might help you"],
         "context_set": ""
        },
        {"tag": "Departments",
         "patterns": ["Tell me the number of departments of Brac University", "I want to know about departments of Brac University", "Departments"],
         "responses": ["Brac University currently holds seven departments. For more information visit https://www.bracu.ac.bd/academics/departments"],
         "context_set": ""
        },
        {"tag": "Expenditure",
         "patterns": ["Do you know how much it cost for studying in Brac University","How much it cost in undergraduate of CSE dept?", "Tell me the financial cost for undergraduate and post graduate program", "cost", "Expenditure"],
         "responses": ["Visit https://www.bracu.ac.bd/fees-and-payment"],
         "context_set": "" 
        },
 
        {"tag": "buildingscafe",
         "patterns": ["Where is building 4", "how many buildings are there in bracu", "how many cafeterias are there in bracu", "where is cafeteria 3 located", "building", "cafeteria"],
         "responses": ["BRACU has it's temporary campus in Mohakhali which consists 8 buildings total. UB1-2 are in one side whereas UB3-8 are in the opposite side. UB1 & UB6 has offices where you can get any information or help. Also the 3 cafeterias are located in UB2, UB3(Ground Floor) and UB6 3rd floor"],
         "context_set": ""
        },

        {"tag": "hours",
         "patterns": ["do you work", "what are your working hours", "when are you active"],
         "responses": ["BRACU BRO is here for you 24/7", "I am here 24 hours a day 7 days a week"],
         "context_set": ""
        },

        {"tag": "office",
         "patterns": ["Could you give me phone number or email id of office", "Give me your contact no","How can I reach u","How can I Talk to you", "email", "contact"],
         "responses": ["Brac Univerity Mobile Number: 01708812609 and Email: info@bracu.ac.bd"],
         "context_set": ""
        }
   ]
}

Overwriting content.json


In [ ]:
import nltk
nltk.download('punkt')

import nltk.stem as stemmer

!pip install tflearn

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 107 kB 15.2 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=cc3b6925c8f3d1373e25f0b931310a2573dbc2979f69a8181574db1e73e385c4
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:

from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()

import numpy as np

import tflearn

import tensorflow as tf

import random

import json

import pickle

try:

  with open("data.pickle", "rb") as f:

    words, labels, training, output = pickle.load(f)

except:  

  with open("content.json") as file:

    data = json.load(file)

  words = []

  labels = []

  docs_x = []

  docs_y = []

  for intent in data["intents"]:

    for pattern in intent["patterns"]:

      wrds = nltk.word_tokenize(pattern)

      words.extend(wrds)

      docs_x.append(wrds)

      docs_y.append(intent["tag"])

    if intent["tag"] not in labels:

      labels.append(intent["tag"])

  words = [stemmer.stem(w.lower()) for w in words if w != "?"]

  words = sorted(list(set(words)))

  labels = sorted(labels)

  training = []

  output = []

  out_empty = [0 for _ in range(len(labels))]

  for x, doc in enumerate(docs_x):

    bag = []

    wrds = [stemmer.stem(w) for w in doc]

    for w in words:

      if w in wrds:

        bag.append(1)

      else:

        bag.append(0)

    output_row = out_empty[:]
    
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)

    output.append(output_row)

  training = np.array(training)

  output = np.array(output)











  training = np.array(training)

  output = np.array(output)

  with open("data.pickle", "wb") as f:

    pickle.dump((words, labels, training, output), f)

#tensorflow.reset_default_graph()

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])

net = tflearn.fully_connected(net, 8)

net = tflearn.fully_connected(net, 8)

net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")

net = tflearn.regression(net)

model = tflearn.DNN(net)


#try:
    #model.load("model.tflearn")
#except:

  #model.fit(training, output, n_epoch = 1000, batch_size = 8, show_metric = True)

  #model.save("model.tflearn")


model.fit(training, output, n_epoch = 1000, batch_size = 8, show_metric = True)

model.save("model.tflearn")


Training Step: 8999  | total loss: 0.01364 | time: 0.058s
| Adam | epoch: 1000 | loss: 0.01364 - acc: 1.0000 -- iter: 64/66
Training Step: 9000  | total loss: 0.01364 | time: 0.074s
| Adam | epoch: 1000 | loss: 0.01364 - acc: 1.0000 -- iter: 66/66
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
#nix

def bag_of_words(s,words):

  bag = [0 for _ in range(len(words))]  
  
  s_words = nltk.word_tokenize(s)
  
  s_words = [stemmer.stem(word.lower())for word in s_words] 
  
  for se in s_words:
  
    for i,w in enumerate(words):
  
      if w==se:
  
        bag[i] = 1

  return np.array(bag)

def chat():

  print("Start talking with the bot. You can ask the basic questions regarding BRAC University to the bot. Since it's a simple bot, there are some limitations. Sometimes it can be possible that you won't get your expected answer. However, it's not too bad though. ENJOY!! (Type quit to end chat with bot!)")

  while True:

    inp=input("You: ")

    if inp.lower()=="quit":

      print("BRACU BRO: See you around bro!!")

      break

    results = model.predict([bag_of_words(inp,words)])[0]

    results_index= np.argmax(results)

    tag = labels[results_index]

    if results[results_index] > 0.7:

      for tg in data["intents"]:

        if tg['tag'] == tag:

          responses = tg['responses']

      print("BRACU BRO: ", end = "")    

      print(random.choice(responses))

    else:

      print("BRACU BRO: I didn't get that bro. Try again!!")

chat()

Start talking with the bot. You can ask the basic questions regarding BRAC University to the bot. Since it's a simple bot, there are some limitations. Sometimes it can be possible that you won't get your expected answer. However, it's not too bad though. ENJOY!! (Type quit to end chat with bot!)
You: Hello
BRACU BRO: Good to see you again!
You: where is bracu?
BRACU BRO: BRAC University (also known as BracU) is a private research university located at 66 Mohakhali, Dhaka, Bangladesh. It was established in 2001 as a branch of Sir Fazle Hasan Abed's BRAC under the Private University Act 1992. [Source: Wikipedia]
You: Departments
BRACU BRO: Brac University currently holds seven departments. For more information visit https://www.bracu.ac.bd/academics/departments
You: Who is pewdiepie?
BRACU BRO: I didn't get that bro. Try again!!
You: quit
BRACU BRO: See you around bro!!
